In [ ]:
#importa bibliotecas
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, f1_score, confusion_matrix
import xgboost as xgb
from xgboost.sklearn import XGBClassifier

In [ ]:
#leitura do train com labels
train = pd.read_csv('train_values.csv')
labels = pd.read_csv('train_labels.csv')

In [ ]:
#separar variaveis categoricas
dt_text = train[['land_surface_condition',
        'foundation_type',
        'roof_type',
        'ground_floor_type',
        'other_floor_type',
        'position',
        'plan_configuration',
         'legal_ownership_status']]


del train['land_surface_condition']
del train['foundation_type']
del train['roof_type']
del train['ground_floor_type']
del train['other_floor_type']
del train['position']
del train['plan_configuration']
del train['legal_ownership_status']
del train['building_id']

In [ ]:
#treino e label
treino = train
label = labels['damage_grade']

In [ ]:
#juntar aplicar get)dummies nas variaveis categoricas e juntar com as outras variaveis
dt_gtdummies = pd.get_dummies(dt_text)
dta = pd.merge(treino,dt_gtdummies,left_index=True, right_index=True)

In [ ]:
#normalizar as variaveis e aplicar get_dummies no label
norma = MinMaxScaler(feature_range=(0,1))
features_norma = norma.fit_transform(dta)
classe = pd.get_dummies(label)

In [ ]:
#separar treino e teste
base_treino, base_teste, classes_treino, classes_teste = train_test_split(features_norma, label, test_size=0.1, random_state = 3)

In [ ]:
#parametizar o modelo
xgb1 = XGBClassifier(
 learning_rate =0.040261492002306595,
 n_estimators=1000,
 max_depth=10,
 min_child_weight=2,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'binary:logistic',
 nthread=4,
 scale_pos_weight=1,
 seed=27)

In [ ]:
#treino
xgb1.fit(base_treino, classes_treino)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.8, gamma=0,
              learning_rate=0.040261492002306595, max_delta_step=0,
              max_depth=10, min_child_weight=2, missing=None, n_estimators=1000,
              n_jobs=1, nthread=4, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=27,
              silent=None, subsample=0.8, verbosity=1)

In [ ]:
#previsão na base de teste
predi= xgb1.predict(base_teste)

In [ ]:
#acuracia
accuracy_score(classes_teste, predi)

0.7530025708913702

In [ ]:
#leitura teste
teste = pd.read_csv('test_values.csv')

In [ ]:
#Tratar variaveis categoricas
dt_test = teste[['land_surface_condition',
        'foundation_type',
        'roof_type',
        'ground_floor_type',
        'other_floor_type',
        'position',
        'plan_configuration',
         'legal_ownership_status']]

del teste['land_surface_condition']
del teste['foundation_type']
del teste['roof_type']
del teste['ground_floor_type']
del teste['other_floor_type']
del teste['position']
del teste['plan_configuration']
del teste['legal_ownership_status']

building_id = teste['building_id']
del teste['building_id']

In [ ]:
#aplicar o get_dummies
dt_gtdummies = pd.get_dummies(dt_test)

In [ ]:
#juntar variaveis de texto e categoricas
dtb = pd.merge(teste,dt_gtdummies,left_index=True, right_index=True)

In [ ]:
#aplicado o modelo
features_tst = norma.fit_transform(dtb)
y_subs = xgb1.predict(features_tst)

In [ ]:
#salvar o arquivo para submeter
saida = pd.DataFrame({'building_id':building_id, 'damage_grade':y_subs})
saida.to_csv('submission_format.csv',index=False)

In [ ]:
#Nome da Competição: Richter's Predictor: Modeling Earthquake Damage
#posição 95 de 3582 competidores (03/12/2020)
#